In [2]:
import os
import requests
import re
import scrapy
import pandas as pd

In [ ]:
#os.getcwd
#os.chdir

In [91]:
#Use this to test xpath/regex

# Test URLs:
url1 = https://boardgamegeek.com/browse/boardgame/page/1
url2 = https://boardgamegeek.com/browse/boardgame/page/191
url3 = https://boardgamegeek.com/boardgame/174430/gloomhaven/credits

from scrapy.http import TextResponse

r = requests.get('https://boardgamegeek.com/browse/boardgame/page/1')
response = TextResponse(r.url, body=r.text, encoding='utf-8')
response

<200 https://boardgamegeek.com/browse/boardgame/page/1>

In [97]:
# dont forget to close the request!
r.close()

In [ ]:
import scrapy
from Capstone.items import CreditItem
import requests
import re

min_players_pattern = r'minplayers":"\d*'
max_players_pattern = r'maxplayers":"\d*'
min_play_time_pattern = r'minplaytime":"\d*'
max_play_time_pattern = r'maxplaytime":"\d*'
age_pattern = r'minage":"\d*'
mechanisms_pattern = r'\\/boardgamemechanic\\/([^\"]+)'
category_pattern = r'\\/boardgamecategory\\/([^\"]+)'
family_pattern = r'\\/boardgamefamily\\/([^\"]+)'
weight_pattern = r'"averageweight":\d\.\d*'

class CreditSpider(scrapy.Spider):
    name = "credit"

    def start_requests(self):
        urls = ['https://boardgamegeek.com/boardgame/174430/gloomhaven/credits',
                'https://boardgamegeek.com/boardgame/174430/gloomhaven/credits',
                'https://boardgamegeek.com/boardgame/161936/pandemic-legacy-season-1/credits',
                'https://boardgamegeek.com/boardgame/38453/space-alert/credits',
                'https://boardgamegeek.com/boardgame/95064/ascension-return-fallen/credits',
                'https://boardgamegeek.com/boardgame/221965/fox-forest/credits'
               ]
        
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
            
    def parse(self, response):
        item = CreditItem()
        item['url'] = response.url
        item['min_players'] = re.findall(min_players_pattern, response.text)
        item['max_players'] = re.findall(max_players_pattern, response.text)
        item['min_play_time'] = re.findall(min_play_time_pattern, response.text)
        item['max_play_time'] = re.findall(max_play_time_pattern, response.text)
        item['age'] = re.findall(age_pattern, response.text)
        item['weight'] = re.findall(weight_pattern, response.text)
        item['category'] = re.findall(category_pattern, response.text)
        item['mechanisms'] = re.findall(mechanisms_pattern, response.text)
        item['family'] = re.findall(family_pattern, response.text)
        yield item

In [2]:
# create item used to structure data 
import scrapy

class BrowseItem(scrapy.Item):
    board_game_rank = scrapy.Field()
    title = scrapy.Field()
    game_sub_url = scrapy.Field()
    geek_rating = scrapy.Field()
    avg_rating = scrapy.Field()
    num_voters = scrapy.Field()
    pass

class CreditItem(scrapy.Item):
    url = scrapy.Field()
    min_players = scrapy.Field()
    max_players = scrapy.Field()
    min_play_time = scrapy.Field()
    max_play_time = scrapy.Field()
    age = scrapy.Field()
    weight = scrapy.Field()
    category = scrapy.Field()
    mechanisms = scrapy.Field()
    family = scrapy.Field()
    pass

In [8]:
import scrapy
from Capstone.items import BrowseItem
import requests

class BrowseSpider(scrapy.Spider):
    name = "browse"

    def start_requests(self):
        urls = ['https://boardgamegeek.com/browse/boardgame/page/1']
        
        # scrapes npages - will work up through page 191 until N/A values become present
        npages = 1

        for i in range(1, npages):
            urls.append("https://boardgamegeek.com/browse/boardgame/page/"+str(i+1)+"")

        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)

    def parse(self, response):
            n = 0
            y = 0
            while n < 100:
                for sel in response.xpath('//table'):
                    item = BrowseItem()
                    item['board_game_rank'] = response.xpath('//tr/td/a[@name]')[n].extract()
                    item['title'] = response.xpath('//tr/td/div/a[@href]/text()')[n].extract()
                    item['game_sub_url'] = response.xpath('//tr/td/div/a/@href')[n].extract()
                    item['geek_rating'] = response.xpath('//tr/td[contains(@class,"collection_bggrating")]/text()')[y].extract()
                    item['avg_rating'] = response.xpath('//tr/td[contains(@class,"collection_bggrating")]/text()')[y+1].extract()
                    item['num_voters'] = response.xpath('//tr/td[contains(@class,"collection_bggrating")]/text()')[y+2].extract()
                    n += 1
                    y += 3
                    yield item
           

In [ ]:
#os.getcwd()
#os.chdir('..')
#this will export in CLI
#!scrapy crawl browse -o browse.csv
#!scrapy crawl credit -o credit.csv

In [3]:
browse = pd.read_csv('browse.csv')

In [49]:
browse.head()

,avg_rating,board_game_rank,game_sub_url,geek_rating,num_voters,title
0,8.83,"<a name=""1""></a>",/boardgame/174430/gloomhaven,8.573,35473,Gloomhaven
1,8.62,"<a name=""2""></a>",/boardgame/161936/pandemic-legacy-season-1,8.472,37381,Pandemic Legacy: Season 1
2,7.45,"<a name=""201""></a>",/boardgame/38453/space-alert,7.247,14338,Space Alert
3,7.30,"<a name=""701""></a>",/boardgame/95064/ascension-return-fallen,6.695,3080,Ascension: Return of the Fallen
4,7.29,"<a name=""401""></a>",/boardgame/221965/fox-forest,6.969,6974,The Fox in the Forest


In [63]:
browse.describe()

,avg_rating,geek_rating,num_voters
count,19021.000000,19021.000000,19021.000000
mean,6.386351,5.690198,813.579465
std,0.926287,0.370845,3331.389788
min,1.050000,3.537000,30.000000
25%,5.800000,5.511000,55.000000
50%,6.420000,5.551000,120.000000
75%,7.000000,5.689000,379.000000
max,9.690000,8.573000,94727.000000


In [4]:
# Creates a column 'full_game_url' to extract a list of urls to be used with the second spyder

browse['full_game_url'] = 'https://boardgamegeek.com' + browse['game_sub_url'].astype(str) + "/credits"

In [76]:
browse.head()

,avg_rating,board_game_rank,game_sub_url,geek_rating,num_voters,title,full_game_url
0,8.83,"<a name=""1""></a>",/boardgame/174430/gloomhaven,8.573,35473,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...
1,8.62,"<a name=""2""></a>",/boardgame/161936/pandemic-legacy-season-1,8.472,37381,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...
2,7.45,"<a name=""201""></a>",/boardgame/38453/space-alert,7.247,14338,Space Alert,https://boardgamegeek.com/boardgame/38453/spac...
3,7.30,"<a name=""701""></a>",/boardgame/95064/ascension-return-fallen,6.695,3080,Ascension: Return of the Fallen,https://boardgamegeek.com/boardgame/95064/asce...
4,7.29,"<a name=""401""></a>",/boardgame/221965/fox-forest,6.969,6974,The Fox in the Forest,https://boardgamegeek.com/boardgame/221965/fox...


In [15]:
full_game_url_list = list(browse['full_game_url'])
full_game_url_list[0:5]

['https://boardgamegeek.com/boardgame/174430/gloomhaven/credits',
 'https://boardgamegeek.com/boardgame/161936/pandemic-legacy-season-1/credits',
 'https://boardgamegeek.com/boardgame/38453/space-alert/credits',
 'https://boardgamegeek.com/boardgame/95064/ascension-return-fallen/credits',
 'https://boardgamegeek.com/boardgame/221965/fox-forest/credits']